In [1]:
!pip install pandas
!pip install openpyxl

Access is denied.
Access is denied.


In [2]:
import pandas as pd

In [3]:
general_information = pd.read_excel('data/general_information_bkp.xlsx')
type_and_level = pd.read_excel('data/type_and_level_bk.xlsx')
geographical_facilities = pd.read_excel('data/geographical_facilities_bkp.xlsx')
buildings_and_library = pd.read_excel('data/buildings_and_library.xlsx')
buildings = pd.read_excel('data/buildings_bkp.xlsx')
compliance1 = pd.read_excel('data/compliance_1_bkp.xlsx')
compliance2 = pd.read_excel('data/compliance_2_bkp.xlsx')
compliance3 = pd.read_excel('data/compliance_3_bkp.xlsx')
compliance4 = pd.read_excel('data/compliance_4_bkp.xlsx')


In [43]:
TYPE_SCHOOL = 1
TYPE_SCHOOL_AND_COLLEGE = 2
TYPE_COLELGE = 3
TYPE_MADRASHA = 4

In [109]:
from enum import Enum

class InstitutionType(Enum):
    SCHOOL = 1
    SCHOOL_AND_COLLEGE = 2
    MADRASHA = 3

In [110]:
from typing import Dict
institute_type_map_total_institution: Dict[int, int] = {}
institute_type_map_total_funded_institution: Dict[int, int] = {}
institute_type_map_total_refunded_institution: Dict[int, int] = {}

In [111]:
merged_eiin_institutes = pd.merge(general_information, type_and_level, on='eiin', how='inner')

In [136]:
df_total_schools = merged_eiin_institutes[merged_eiin_institutes['institution_type']==TYPE_SCHOOL]
df_total_school_and_college= merged_eiin_institutes[merged_eiin_institutes['institution_type'].isin([TYPE_SCHOOL_AND_COLLEGE, TYPE_COLELGE])]
df_total_madrasha = merged_eiin_institutes[merged_eiin_institutes['institution_type']== TYPE_MADRASHA]

In [137]:
institute_type_map_total_institution[TYPE_SCHOOL] = df_total_schools.shape[0]
institute_type_map_total_institution[TYPE_SCHOOL_AND_COLLEGE] = df_total_school_and_college.shape[0]
institute_type_map_total_institution[TYPE_MADRASHA] = df_total_madrasha.shape[0]

In [138]:
pbgsi_institutes = compliance2[(compliance2['pbgsi_scheme']==1)]

total_pgbsi_institutes = pbgsi_institutes.shape[0]

In [139]:
general_information_pbgsi = pd.merge(general_information, pbgsi_institutes, on="eiin", how="inner")

In [140]:
type_and_level_secondary_school_secondary = type_and_level[type_and_level['institution_type']==TYPE_SCHOOL]

In [141]:
general_information_pbgsi_secondary = pd.merge(general_information_pbgsi, type_and_level_secondary_school_secondary, on = "eiin", how="inner")

In [142]:
total_secondary_school_pbgsi = general_information_pbgsi_secondary.shape[0]
institute_type_map_total_funded_institution[TYPE_SCHOOL] = total_secondary_school_pbgsi

In [143]:
type_and_level_secondary_school_and_college = type_and_level[type_and_level["institution_type"].isin([TYPE_SCHOOL_AND_COLLEGE, TYPE_COLELGE]) ]

In [144]:
general_information_pbgsi_school_and_college = pd.merge(general_information_pbgsi, type_and_level_secondary_school_and_college, on='eiin', how='inner')

In [145]:
total_school_and_college = general_information_pbgsi_school_and_college.shape[0]
institute_type_map_total_funded_institution[TYPE_SCHOOL_AND_COLLEGE] = total_school_and_college

In [146]:
type_and_level_madrasha = type_and_level[type_and_level['institution_type']==TYPE_MADRASHA]

In [147]:
general_information_pbgsi_madrasha = pd.merge(general_information_pbgsi, type_and_level_madrasha, on='eiin', how='inner')

In [148]:
total_madrasha = general_information_pbgsi_madrasha.shape[0]
institute_type_map_total_funded_institution[TYPE_MADRASHA] = total_madrasha

In [149]:
import openpyxl
from openpyxl.utils import get_column_letter,get_column_letter
from openpyxl.styles import Alignment, Border, Side

In [150]:
wb = openpyxl.Workbook()
ws = wb.active

ws['A1'] = 'Type of Institute'
ws.merge_cells('A1:A2')

ws ['B1'] = 'Received Grant'
ws.merge_cells('B1:C1')

ws['D1'] = 'Refunded Grant'
ws.merge_cells('D1:E1')

ws['B2'] = 'Number of Institute'
ws['C2'] = 'Percentage (%)'

ws['D2']= 'Number of Institute'
ws['E2'] = 'Percentage (%)'

row_num = 3

for inst_type in InstitutionType:
    institution_type_cell = "A"+str(row_num)
    ws[institution_type_cell] = inst_type.name


    total_funded_institution_count = institute_type_map_total_funded_institution.get(inst_type.value, 0)
    total_funded_instititon_count_cell = "B"+str(row_num)
    ws[total_funded_instititon_count_cell] = str(total_funded_institution_count)

    total_institution_count = institute_type_map_total_institution.get(inst_type.value)
    total_fund_percentage:float = 0.0 if total_funded_institution_count==0 else  total_funded_institution_count/total_institution_count
    total_fund_percentage_cell = "C"+str(row_num)
    ws[total_fund_percentage_cell] = str(total_fund_percentage)

    row_num+=1

center_alignment = Alignment(horizontal='center', vertical='center')
thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

for row in ws.iter_rows():
    for cell in row:
        cell.alignment = center_alignment
        cell.border = thin_border

for column_cells in ws.columns:
    length = max(len(str(cell.value)) if cell.value is not None else 0 for cell in column_cells)
    column_letter = get_column_letter(column_cells[0].column)
    ws.column_dimensions[column_letter].width = length + 2  # Add some padding
    
wb.save('output/table_1_number_of_institutes_having_grand_and_refund.xlsx')